<a href="https://colab.research.google.com/github/NearANDfar13/NLP_Winter_2024/blob/main/Final_Notebooks_San_Fran/NAA_San_Fran_NER_Analysis_ImpressoAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

#From NLP class on Impresso API
import requests

def get_linked_entities(text, coarse_only=False):
    """
    Calls the external API to get named entity recognition (NER) results.
    """
    url = "https://impresso-annotation.epfl.ch/api/ner/"
    payload = {"data": text}
    try:
        response = requests.post(url, json=payload)
        if response.status_code == 200:
            data = response.json()
            data["text"] = text
            # remove fine-grained and components
            if coarse_only:
                for ne in data["nes"]:
                    data["nes"] = [ne for ne in data["nes"] if not "." in ne["type"]]
            return data
        else:
            print(f"Request failed with status code {response.status_code}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None


In [ ]:
!git clone https://github.com/NearANDfar13/NLP_Winter_2024.git

Cloning into 'NLP_Winter_2024'...
remote: Enumerating objects: 188, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 188 (delta 15), reused 2 (delta 2), pack-reused 150 (from 1)
Receiving objects: 100% (188/188), 11.52 MiB | 31.45 MiB/s, done.
Resolving deltas: 100% (84/84), done.


Chunk 1: 0-50

In [ ]:
#Data organizing and filtering method from NLP class
#Import the dataset "San Fran EExtended Sample"
import pandas as pd

articles_df = pd.read_excel('/content/NLP_Winter_2024/NA_San_Fran_Extended_Sample.xlsx')

#Display DataFrame

articles_df = articles_df[:50]
articles_df.head()

,ID,Day,Short_Name,Full_Name,Place,Frequency,Publication,Text,Page,Link,Datengeber
0,3043262-5,19/04/1906,Annener Zeitung,"Annener Zeitung, verbunden mit der Annener Vol...","Annen (Witten), Witten-Annen",NaN,1885-1943,"New=York, 14 Francisco von eine gesucht. Tause...",2.,https://www.deutsche-digitale-bibliothek.de/ne...,Westfälische Wilhelms-Universität Münster Univ...
1,3118927-1,19/04/1906,Anzeiger vom Oberland,"Anzeiger vom Oberland, Tageszeitung für das Ob...",Biberach an der Riß,täglich,1870-1932,"au- Oakland als Gerücht gemeldet, daß da- amer...",2.,https://www.deutsche-digitale-bibliothek.de/ne...,Kreisarchiv des Landkreises Biberach
2,3118927-1,19/04/1906,Anzeiger vom Oberland,"Anzeiger vom Oberland, Tageszeitung für das Ob...",Biberach an der Riß,täglich,1870-1932,"ertappt wird, zu erschießen. ir Gf Store (CaUf...",6.,https://www.deutsche-digitale-bibliothek.de/ne...,Kreisarchiv des Landkreises Biberach
3,2797055-3,19/04/1906,Badische Presse,Badische Presse : Generalanzeiger der Residenz...,Karlsruhe,täglich,1890-1944,Die Marinestationen am Süllen Ozean versuchten...,4.,https://www.deutsche-digitale-bibliothek.de/ne...,Badische Landesbibliothek
4,2958285-4,19/04/1906,Der Beobachter,"Der Beobachter, ein Volksblatt aus Schwaben","Stuttgart, Württemberg",täglich,1833-1920,"Erdbeben in Ealifornien. New-Sjork, 18. April....",3.,https://www.deutsche-digitale-bibliothek.de/ne...,Württembergische Landesbibliothek


## Named Entity Extraction

In [ ]:
import requests
import pandas as pd
import json


def process_dataframe(df):
    results = []
    for index, row in df.iterrows():
        text = row['Text']
        api_result = get_linked_entities(text, coarse_only=True)
        results.append(api_result)
    df['NER_results'] = results
    return df

articles_df = process_dataframe(articles_df)
articles_df.head()

,ID,Day,Short_Name,Full_Name,Place,Frequency,Publication,Text,Page,Link,Datengeber,NER_results
0,3043262-5,19/04/1906,Annener Zeitung,"Annener Zeitung, verbunden mit der Annener Vol...","Annen (Witten), Witten-Annen",NaN,1885-1943,"New=York, 14 Francisco von eine gesucht. Tause...",2.,https://www.deutsche-digitale-bibliothek.de/ne...,Westfälische Wilhelms-Universität Münster Univ...,"{'ts': '2025-03-15T00:32:42Z', 'sys_id': 'ner-..."
1,3118927-1,19/04/1906,Anzeiger vom Oberland,"Anzeiger vom Oberland, Tageszeitung für das Ob...",Biberach an der Riß,täglich,1870-1932,"au- Oakland als Gerücht gemeldet, daß da- amer...",2.,https://www.deutsche-digitale-bibliothek.de/ne...,Kreisarchiv des Landkreises Biberach,"{'ts': '2025-03-15T00:32:54Z', 'sys_id': 'ner-..."
2,3118927-1,19/04/1906,Anzeiger vom Oberland,"Anzeiger vom Oberland, Tageszeitung für das Ob...",Biberach an der Riß,täglich,1870-1932,"ertappt wird, zu erschießen. ir Gf Store (CaUf...",6.,https://www.deutsche-digitale-bibliothek.de/ne...,Kreisarchiv des Landkreises Biberach,"{'ts': '2025-03-15T00:33:07Z', 'sys_id': 'ner-..."
3,2797055-3,19/04/1906,Badische Presse,Badische Presse : Generalanzeiger der Residenz...,Karlsruhe,täglich,1890-1944,Die Marinestationen am Süllen Ozean versuchten...,4.,https://www.deutsche-digitale-bibliothek.de/ne...,Badische Landesbibliothek,"{'ts': '2025-03-15T00:33:30Z', 'sys_id': 'ner-..."
4,2958285-4,19/04/1906,Der Beobachter,"Der Beobachter, ein Volksblatt aus Schwaben","Stuttgart, Württemberg",täglich,1833-1920,"Erdbeben in Ealifornien. New-Sjork, 18. April....",3.,https://www.deutsche-digitale-bibliothek.de/ne...,Württembergische Landesbibliothek,"{'ts': '2025-03-15T00:33:45Z', 'sys_id': 'ner-..."


In [ ]:
#extract locations

import pandas as pd

def extract_locations(df):
    places = []
    for index, row in df.iterrows():
        ner_results = row['NER_results']
        if ner_results and 'nes' in ner_results:
            # Use 'surface' instead of 'text' to get the entity text
            location_names = [ne['surface'] for ne in ner_results['nes'] if ne.get('type') == 'loc']
            places.append(', '.join(location_names))  # Join multiple locations
        else:
            places.append('')  # Handle cases where NER_results is None or nes key is missing
    df['places'] = places
    return df

articles_df = extract_locations(articles_df)
print(articles_df[['places']])

                                               places
0   New=York, New=York, Francisco, San Frane, Form...
1   Oakland, Oakland, Bucht von San Franc'Sco, New...
2   CaUfornten, Nrw, Nevada, N-w Uork, renz, renz,...
3   Süllen Ozean, San Francisco, San Francisco, Sa...
4   Ealifornien, New-Sjork, Kansas Cith, Los Angel...
5   San Francisco, San Francisco, San Francisco, S...
6   SanFrancisco, SanFrancisco, SanFrancisco, SanF...
7   San Francisco, San Francisco, San Francisco, S...
8   San Francisco, San Francisco, San Francisco, S...
9   Cincinnati, Oakland, Oakland, Stillen Ozeans, ...
10  Newyork, Oakland, San Francisco, San Francisco...
11  Kaliforniens, San Francisco, New Pork, San Fra...
12  San Francisco, San Francisco, San Francisco, S...
13  San Francisco, San Francisco, San Francisco, S...
14  Santa Cruz, Agnew, San Joss, San Francisco, Sa...
15  San Francisco, San Francisco, San Francisco, S...
16  Los Angelos, San Francisco, Marketstraße, Trüm...
17  Newyork, Newyork, Newyor

In [ ]:
# export as excel file
articles_df.to_excel('earthquake_articles_locations.xlsx', index=False)

Chunk 2: 50-100

In [ ]:
#Import second part of dataset "San Fran EExtended Sample"
articles_df2 = pd.read_excel('/content/NLP_Winter_2024/NA_San_Fran_Extended_Sample.xlsx')

#Display DataFrame

articles_df2 = articles_df2[50:100]
articles_df2.head()

,ID,Day,Short_Name,Full_Name,Place,Frequency,Publication,Text,Page,Link,Datengeber
50,3048191-0,19/04/1906,Rheinischer Merkur,"Rheinischer Merkur, Kölnische Landeszeitung",Köln,dreimal wöchentlich,1881-1922,"000 angegeben wird, so ist dies nur durch den ...",2.,https://www.deutsche-digitale-bibliothek.de/ne...,Universitäts- und Landesbibliothek der Rheinis...
51,2928761-3,19/04/1906,Rheinisches Volksblatt,"Rheinisches Volksblatt, Hildener Zeitung und T...","Düsseldorf, Mettmann, Solingen, Hilden",dreimal wöchentlich,1883-1941,Trümmerhaufen versperrt. Später eingetroffene ...,2.,https://www.deutsche-digitale-bibliothek.de/ne...,Universitäts- und Landesbibliothek der Rheinis...
52,3123872-5,19/04/1906,Riedlinger Zeitung,"Riedlinger Zeitung, Tag- und Anzeigeblatt für ...",Riedlingen,dreimal wöchentlich,1848-1933,"haben, jeden, der beim Diebstahl ertappt wird,...",2.,https://www.deutsche-digitale-bibliothek.de/ne...,Württembergische Landesbibliothek
53,3123872-5,19/04/1906,Riedlinger Zeitung,"Riedlinger Zeitung, Tag- und Anzeigeblatt für ...",Riedlingen,dreimal wöchentlich,1848-1933,"11 Mann getötet, 50 zum teil tötlich verletzt....",3.,https://www.deutsche-digitale-bibliothek.de/ne...,Württembergische Landesbibliothek
54,2973532-4,19/04/1906,Riesaer Tageblatt,"Riesaer Tageblatt und Anzeiger, (Elbeblatt und...",Riesa,"wöchentlich, dreimal wöchentlich",1893-1945,folgedessen ins Meer gesunken. Aus' Sacramentv...,3.,https://www.deutsche-digitale-bibliothek.de/ne...,Sächsische Landesbibliothek - Staats- und Univ...


In [ ]:
#NER of second part of dataset "San Fran Extended Sample"
articles_df2 = process_dataframe(articles_df2)
articles_df2.head()

,ID,Day,Short_Name,Full_Name,Place,Frequency,Publication,Text,Page,Link,Datengeber,NER_results
50,3048191-0,19/04/1906,Rheinischer Merkur,"Rheinischer Merkur, Kölnische Landeszeitung",Köln,dreimal wöchentlich,1881-1922,"000 angegeben wird, so ist dies nur durch den ...",2.,https://www.deutsche-digitale-bibliothek.de/ne...,Universitäts- und Landesbibliothek der Rheinis...,"{'ts': '2025-03-15T01:00:02Z', 'sys_id': 'ner-..."
51,2928761-3,19/04/1906,Rheinisches Volksblatt,"Rheinisches Volksblatt, Hildener Zeitung und T...","Düsseldorf, Mettmann, Solingen, Hilden",dreimal wöchentlich,1883-1941,Trümmerhaufen versperrt. Später eingetroffene ...,2.,https://www.deutsche-digitale-bibliothek.de/ne...,Universitäts- und Landesbibliothek der Rheinis...,"{'ts': '2025-03-15T01:00:07Z', 'sys_id': 'ner-..."
52,3123872-5,19/04/1906,Riedlinger Zeitung,"Riedlinger Zeitung, Tag- und Anzeigeblatt für ...",Riedlingen,dreimal wöchentlich,1848-1933,"haben, jeden, der beim Diebstahl ertappt wird,...",2.,https://www.deutsche-digitale-bibliothek.de/ne...,Württembergische Landesbibliothek,"{'ts': '2025-03-15T01:00:17Z', 'sys_id': 'ner-..."
53,3123872-5,19/04/1906,Riedlinger Zeitung,"Riedlinger Zeitung, Tag- und Anzeigeblatt für ...",Riedlingen,dreimal wöchentlich,1848-1933,"11 Mann getötet, 50 zum teil tötlich verletzt....",3.,https://www.deutsche-digitale-bibliothek.de/ne...,Württembergische Landesbibliothek,"{'ts': '2025-03-15T01:00:30Z', 'sys_id': 'ner-..."
54,2973532-4,19/04/1906,Riesaer Tageblatt,"Riesaer Tageblatt und Anzeiger, (Elbeblatt und...",Riesa,"wöchentlich, dreimal wöchentlich",1893-1945,folgedessen ins Meer gesunken. Aus' Sacramentv...,3.,https://www.deutsche-digitale-bibliothek.de/ne...,Sächsische Landesbibliothek - Staats- und Univ...,"{'ts': '2025-03-15T01:00:38Z', 'sys_id': 'ner-..."


In [ ]:
#extract locations of second part of dataset

articles_df2 = extract_locations(articles_df2)
print(articles_df2[['places']])

                                               places
50  San Francisco, San Francisco, San Francisco, S...
51  San Francisco, San Francisco, San Francisco, S...
52  San Francisco, San Francisco, San Francisco, S...
53  Washington, San Francisco, San Francisco, San ...
54  Santa Rosa, Ackland, San Franzisko, San Franzi...
55  San Francisco, San Francisco, San Francisco, S...
56  Bucht, San Francisco, San Francisco, San Franc...
57  San Francisco, San Francisco, Kalifornien, Kal...
58  Saciamento, San Francisco, San Francisco, San ...
59  San Franzisko, San Franzisko, San Franzisko, S...
60  Oakland, Oakland, Stillen Ozean, Bricht, San F...
61  Stadt San Francisco, San Francisco, San Franci...
62  Santa Rosa, Santa Rosa, San Francisco, San Fra...
63  New=York, New=York, New=York, New=York, New=Yo...
64  San, San, San, San, San, San, San, San, San, S...
65  New=York, New=York, New=York, New=York, San Fr...
66  Kalifornien, Kalifornien, San Francisco, San F...
67  New York, New York, Sa» 

In [ ]:
# export updated second part of dataset as excel file
articles_df2.to_excel('earthquake_articles_locations2.xlsx', index=False)

Chunk 3: 100-200

In [ ]:
#Import third part of dataset "San Fran Extended Sample"
articles_df3 = pd.read_excel('/content/NLP_Winter_2024/NA_San_Fran_Extended_Sample.xlsx')

#Display DataFrame

articles_df3 = articles_df3[100:200]
articles_df3.head()

,ID,Day,Short_Name,Full_Name,Place,Frequency,Publication,Text,Page,Link,Datengeber
100,2956643-5,20/04/1906,Iserlohner Kreisanzeiger,"Iserlohner Kreisanzeiger und Zeitung, märkisch...","Iserlohn, Landkreis Iserlohn",NaN,1898-1950,Die Sanitatswachen der Unfallstation und der F...,3.,https://www.deutsche-digitale-bibliothek.de/ne...,Westfälische Wilhelms-Universität Münster Univ...
101,824611-7,20/04/1906,Jeversches Wochenblatt,"Jeversches Wochenblatt, Friesisches Tageblatt ...",Jever,täglich,1817-1949,"jeden, der beim Diebstahle betroffen «ird, nie...",2.,https://www.deutsche-digitale-bibliothek.de/ne...,Landesbibliothek Oldenburg
102,824611-7,20/04/1906,Jeversches Wochenblatt,"Jeversches Wochenblatt, Friesisches Tageblatt ...",Jever,täglich,1817-1949,des Militärzuges der sibirischen Bahn sind 10 ...,3.,https://www.deutsche-digitale-bibliothek.de/ne...,Landesbibliothek Oldenburg
103,2719361-5,20/04/1906,Kölnische Zeitung,"Kölnische Zeitung, mit Wirtschafts- und Handel...","Köln, Kleve (Kreis Kleve), Jülich",täglich,1803-1945,"Paris, 19. April.(Telegr.) Die Regierung hat b...",2.,https://www.deutsche-digitale-bibliothek.de/ne...,Universitäts- und Landesbibliothek der Rheinis...
104,2719361-5,20/04/1906,Kölnische Zeitung,"Kölnische Zeitung, mit Wirtschafts- und Handel...","Köln, Kleve (Kreis Kleve), Jülich",täglich,1803-1945,schäft bestand heute wieder zum größten Teil a...,6.,https://www.deutsche-digitale-bibliothek.de/ne...,Universitäts- und Landesbibliothek der Rheinis...


In [ ]:
#NER of third part of dataset "San Fran Extended Sample"
articles_df3 = process_dataframe(articles_df3)
articles_df3.head()

,ID,Day,Short_Name,Full_Name,Place,Frequency,Publication,Text,Page,Link,Datengeber,NER_results
100,2956643-5,20/04/1906,Iserlohner Kreisanzeiger,"Iserlohner Kreisanzeiger und Zeitung, märkisch...","Iserlohn, Landkreis Iserlohn",NaN,1898-1950,Die Sanitatswachen der Unfallstation und der F...,3.,https://www.deutsche-digitale-bibliothek.de/ne...,Westfälische Wilhelms-Universität Münster Univ...,"{'ts': '2025-03-15T01:32:40Z', 'sys_id': 'ner-..."
101,824611-7,20/04/1906,Jeversches Wochenblatt,"Jeversches Wochenblatt, Friesisches Tageblatt ...",Jever,täglich,1817-1949,"jeden, der beim Diebstahle betroffen «ird, nie...",2.,https://www.deutsche-digitale-bibliothek.de/ne...,Landesbibliothek Oldenburg,"{'ts': '2025-03-15T01:32:52Z', 'sys_id': 'ner-..."
102,824611-7,20/04/1906,Jeversches Wochenblatt,"Jeversches Wochenblatt, Friesisches Tageblatt ...",Jever,täglich,1817-1949,des Militärzuges der sibirischen Bahn sind 10 ...,3.,https://www.deutsche-digitale-bibliothek.de/ne...,Landesbibliothek Oldenburg,"{'ts': '2025-03-15T01:32:54Z', 'sys_id': 'ner-..."
103,2719361-5,20/04/1906,Kölnische Zeitung,"Kölnische Zeitung, mit Wirtschafts- und Handel...","Köln, Kleve (Kreis Kleve), Jülich",täglich,1803-1945,"Paris, 19. April.(Telegr.) Die Regierung hat b...",2.,https://www.deutsche-digitale-bibliothek.de/ne...,Universitäts- und Landesbibliothek der Rheinis...,"{'ts': '2025-03-15T01:32:59Z', 'sys_id': 'ner-..."
104,2719361-5,20/04/1906,Kölnische Zeitung,"Kölnische Zeitung, mit Wirtschafts- und Handel...","Köln, Kleve (Kreis Kleve), Jülich",täglich,1803-1945,schäft bestand heute wieder zum größten Teil a...,6.,https://www.deutsche-digitale-bibliothek.de/ne...,Universitäts- und Landesbibliothek der Rheinis...,"{'ts': '2025-03-15T01:33:20Z', 'sys_id': 'ner-..."


In [ ]:
#extract locations of third part of dataset

articles_df3 = extract_locations(articles_df3)
print(articles_df3[['places']])

                                                places
100  Kalisernien, Newyork, Newyork, Rew York, New P...
101  San Francisco, San Francisco, San Francisco, S...
102  sibirischen, Kalifornien, San Francisco, Sacra...
103  Paris, Bereinigten, Sau Francisco, Karlsruhe, ...
104  San Francisco, San Francisco, San Francisco, S...
..                                                 ...
195  Bremens, Stadt San Francisco, Stadt San Franci...
196  New Pork, New Pork, San Francisco, San Francis...
197  San Francisco, San Francisco, San Francisco, S...
198  New=York, New=York, San Francisco, San Francis...
199         Golf von Neapel, San Francisco, Courrières

[100 rows x 1 columns]


In [ ]:
# export updated third part of dataset as excel file
articles_df3.to_excel('earthquake_articles_locations3.xlsx', index=False)

Chunk 4: 200-310

In [ ]:
#Import fourth part of dataset "San Fran Extended Sample"
articles_df4 = pd.read_excel('/content/NLP_Winter_2024/NA_San_Fran_Extended_Sample.xlsx')

#Display DataFrame

articles_df4 = articles_df4[200:310]
articles_df4.head()

,ID,Day,Short_Name,Full_Name,Place,Frequency,Publication,Text,Page,Link,Datengeber
200,3000029-4,22/04/1906,Saale-Zeitung,"Saale-Zeitung, allgemeine Zeitung für Mittelde...",Halle (Saale),"täglich, unregelmäßig",1873-1933,"wird auferlegt, um die Ernährung der Zurückble...",2.,https://www.deutsche-digitale-bibliothek.de/ne...,"Martin-Luther-Universität Halle-Wittenberg, Un..."
201,3000029-4,22/04/1906,Saale-Zeitung,"Saale-Zeitung, allgemeine Zeitung für Mittelde...",Halle (Saale),"täglich, unregelmäßig",1873-1933,"ſchwere Brandwunden, daß der Tod ſofort eintra...",3.,https://www.deutsche-digitale-bibliothek.de/ne...,"Martin-Luther-Universität Halle-Wittenberg, Un..."
202,2888222-2,22/04/1906,Sächsische Dorfzeitung,"Sächsische Dorfzeitung und Elbgaupresse, mit L...","Dresden, Blasewitz, Loschwitz, Dresden-Blasewi...",täglich,1905-1933,auch sah er tiefe Risse in der Erde und geiser...,6.,https://www.deutsche-digitale-bibliothek.de/ne...,Sächsische Landesbibliothek - Staats- und Univ...
203,2983003-5,22/04/1906,Sauerländisches Volksblatt,"Sauerländisches Volksblatt, aeltester Anzeiger...",Olpe,NaN,1876-1941,spendete 100000 Mt. * San Franzisko vor der Ze...,2.,https://www.deutsche-digitale-bibliothek.de/ne...,Westfälische Wilhelms-Universität Münster Univ...
204,2983003-5,22/04/1906,Sauerländisches Volksblatt,"Sauerländisches Volksblatt, aeltester Anzeiger...",Olpe,NaN,1876-1941,dem das Wimmern der Verwundeten drang. Zu Tode...,3.,https://www.deutsche-digitale-bibliothek.de/ne...,Westfälische Wilhelms-Universität Münster Univ...


In [ ]:
#NER of fourth part of dataset "San Fran Extended Sample"
articles_df4 = process_dataframe(articles_df4)
articles_df4.head()

,ID,Day,Short_Name,Full_Name,Place,Frequency,Publication,Text,Page,Link,Datengeber,NER_results
200,3000029-4,22/04/1906,Saale-Zeitung,"Saale-Zeitung, allgemeine Zeitung für Mittelde...",Halle (Saale),"täglich, unregelmäßig",1873-1933,"wird auferlegt, um die Ernährung der Zurückble...",2.,https://www.deutsche-digitale-bibliothek.de/ne...,"Martin-Luther-Universität Halle-Wittenberg, Un...","{'ts': '2025-03-15T01:49:07Z', 'sys_id': 'ner-..."
201,3000029-4,22/04/1906,Saale-Zeitung,"Saale-Zeitung, allgemeine Zeitung für Mittelde...",Halle (Saale),"täglich, unregelmäßig",1873-1933,"ſchwere Brandwunden, daß der Tod ſofort eintra...",3.,https://www.deutsche-digitale-bibliothek.de/ne...,"Martin-Luther-Universität Halle-Wittenberg, Un...","{'ts': '2025-03-15T01:49:13Z', 'sys_id': 'ner-..."
202,2888222-2,22/04/1906,Sächsische Dorfzeitung,"Sächsische Dorfzeitung und Elbgaupresse, mit L...","Dresden, Blasewitz, Loschwitz, Dresden-Blasewi...",täglich,1905-1933,auch sah er tiefe Risse in der Erde und geiser...,6.,https://www.deutsche-digitale-bibliothek.de/ne...,Sächsische Landesbibliothek - Staats- und Univ...,"{'ts': '2025-03-15T01:49:20Z', 'sys_id': 'ner-..."
203,2983003-5,22/04/1906,Sauerländisches Volksblatt,"Sauerländisches Volksblatt, aeltester Anzeiger...",Olpe,NaN,1876-1941,spendete 100000 Mt. * San Franzisko vor der Ze...,2.,https://www.deutsche-digitale-bibliothek.de/ne...,Westfälische Wilhelms-Universität Münster Univ...,"{'ts': '2025-03-15T01:49:29Z', 'sys_id': 'ner-..."
204,2983003-5,22/04/1906,Sauerländisches Volksblatt,"Sauerländisches Volksblatt, aeltester Anzeiger...",Olpe,NaN,1876-1941,dem das Wimmern der Verwundeten drang. Zu Tode...,3.,https://www.deutsche-digitale-bibliothek.de/ne...,Westfälische Wilhelms-Universität Münster Univ...,"{'ts': '2025-03-15T01:49:37Z', 'sys_id': 'ner-..."


In [ ]:
#extract locations of fourth part of dataset

articles_df4 = extract_locations(articles_df4)
print(articles_df4[['places']])

                                                places
200  New Hork, New Hork, San Francisco, San Francis...
201  San Franeisco, San Francisco, San Francisco, S...
202  Newyork, San Francisco, San Francisco, San Fra...
203  San Francisco, San Francisco, San Francisco, S...
204  Washington, Washington, San Franzisko, San Fra...
..                                                 ...
305  Berlins, Deulsch-Südwestafrika, Taubcustruße, ...
306  Vereinigten, Amerika, Amerika, Amerika, Stadt ...
307  Vereinigten, Staaten, Staaten, von Amerika, Sa...
308  San Francisco, San Francisco, San Francisco, S...
309  Duisburg, Quisburg, Kalifornien, Neuyork, Neuy...

[110 rows x 1 columns]


In [ ]:
# export updated fourth part of dataset as excel file
articles_df4.to_excel('earthquake_articles_locations4.xlsx', index=False)